In [20]:
import pandas as pd
import rasterio
import xarray as xr
import rioxarray
import numpy as np
import geopandas as gpd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
import time
from shapely.geometry import Polygon

In [2]:
file_path = '/Users/andrea/Downloads/GHS_SMOD_E2010_GLOBE_R2023A_54009_1000_V1_0/GHS_SMOD_E2010_GLOBE_R2023A_54009_1000_V1_0.tif'

In [3]:
a = rioxarray.open_rasterio(file_path)

In [4]:
a = a.squeeze('band', drop=True)

In [5]:
a_stacked = a.stack(z=('x', 'y'))

In [6]:
p = a_stacked.to_pandas().reset_index()

In [7]:
p.rename(columns={0: 'degree_of_urbanization'}, inplace=True)

In [8]:
p = p.loc[p['degree_of_urbanization'] > 13]

In [9]:
p_coords = p[['x', 'y']]

In [10]:
gpd_coords = gpd.GeoDataFrame(geometry=gpd.points_from_xy(p_coords.x, p_coords.y), crs='World_Mollweide')

In [11]:
def create_square(x_center, y_center, length_side):
    half_length_side = length_side / 2
    x_min = x_center - half_length_side
    x_max = x_center + half_length_side
    y_min = y_center - half_length_side
    y_max = y_center + half_length_side
    return Polygon([(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)])

In [12]:
gpd_coords['squares'] = gpd_coords.geometry.apply(lambda pt: create_square(x_center=pt.x, y_center=pt.y, length_side=1000))

In [15]:
sqrs = gpd_coords[['squares']].set_geometry('squares')

In [16]:
sqrs['cluster'] = np.random.randint(0, 100, sqrs.shape[0])

In [17]:
dissolved_sqrs = sqrs.dissolve(by='cluster')

In [31]:
bounds = dissolved_sqrs.bounds

In [32]:
bounds_row_0 = bounds.iloc[0]

In [34]:
w = a.loc[dict(x=slice(bounds_row_0.minx, bounds_row_0.maxx), y=slice(bounds_row_0.maxy, bounds_row_0.miny))]

In [39]:
w_stack = w.stack(z=('x', 'y'))

In [40]:
w_stack = w_stack.to_pandas().reset_index()

In [42]:
w_stack.rename(columns={0: 'degree_of_urbanization'}, inplace=True)

In [43]:
w_stack = w_stack.loc[w_stack['degree_of_urbanization'] > 13]

In [44]:
w_stack_gpd = gpd.GeoDataFrame(geometry=gpd.points_from_xy(w_stack.x, w_stack.y), crs='World_Mollweide')

In [ ]:
j_mask = w_stack_gpd.within(dissolved_sqrs.iloc[0:1].geometry.iloc[0])

In [ ]:
j = w_stack_gpd.loc[j_mask]

In [50]:
j = dissolved_sqrs.iloc[0:1].sjoin(w_stack_gpd, predicate='contains', how='inner')

In [35]:
windows = [rasterio.windows.from_bounds(*row, transform=a.rio.transform()) for _, row in bounds.iterrows()]

In [36]:
b = rasterio.open(file_path)

In [37]:
w1 = b.read(1, window=windows[0])